In [51]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [52]:
%sql sqlite:///library.db

### Create Library Item Table ###

In [53]:
%%sql
DROP TABLE IF EXISTS LibraryItem;
DROP TABLE IF EXISTS Member;
DROP TABLE IF EXISTS Personnel;
DROP TABLE IF EXISTS Event;
DROP TABLE IF EXISTS FutureItem;
DROP TABLE IF EXISTS Register;
DROP TABLE IF EXISTS Borrows;
DROP TRIGGER IF EXISTS set_item_checked_out;
DROP TRIGGER IF EXISTS set_item_available;
DROP TRIGGER IF EXISTS ensure_future_item_pending;
DROP TRIGGER IF EXISTS update_future_item_status;
DROP TRIGGER IF EXISTS calculate_fine_on_return;
DROP TRIGGER IF EXISTS set_item_available_on_return;

Running query in 'sqlite:///library.db'

++
||
++
++

In [54]:
%%sql

CREATE TABLE IF NOT EXISTS LibraryItem (
    itemID INTEGER PRIMARY KEY AUTOINCREMENT,
    title TEXT NOT NULL,
    author TEXT,
    releaseYear INTEGER,
    issueNumber INTEGER,
    itemType TEXT,
    itemStatus TEXT DEFAULT 'Available' CHECK (itemStatus IN ('Available','On-Hold','Checked-Out','Pending'))
);

Running query in 'sqlite:///library.db'

++
||
++
++

### Create Member Table ###

In [55]:
%%sql
DROP TABLE IF EXISTS Member;

Running query in 'sqlite:///library.db'

++
||
++
++

In [56]:
%%sql
CREATE TABLE Member (
    memberID INTEGER PRIMARY KEY AUTOINCREMENT,
    firstName TEXT NOT NULL,
    lastName TEXT NOT NULL,
    phoneNumber TEXT,
    email TEXT,
    membershipType TEXT,
    CHECK (email IS NOT NULL OR phoneNumber IS NOT NULL)
);

Running query in 'sqlite:///library.db'

++
||
++
++

### Create Personnel Table ###

In [57]:
%%sql
CREATE TABLE Personnel (
    personnelID INTEGER,
    firstName TEXT NOT NULL,
    lastName TEXT NOT NULL,
    personnelRole TEXT,
    primaryLocation TEXT,
    salary REAL,
    PRIMARY KEY (personnelID)
)

Running query in 'sqlite:///library.db'

++
||
++
++

### Create Event Table ###

In [58]:
%%sql
CREATE TABLE Event (
    eventID INTEGER,
    eventName TEXT NOT NULL,
    eventType TEXT,
    eventDate DATE NOT NULL,
    eventLocation TEXT,
    audienceType TEXT,
    roomNumber TEXT,
    PRIMARY KEY (eventID)
);

Running query in 'sqlite:///library.db'

++
||
++
++

### Create Future Item Entity Set ###

In [59]:
%%sql
CREATE TABLE FutureItem (
    itemID INTEGER,
    dateOfArrival DATE,
    approvalStatus TEXT DEFAULT 'Pending' CHECK (approvalStatus IN ('Pending','Complete','Denied')),
    FOREIGN KEY (itemID) REFERENCES LibraryItem(itemID)
);

Running query in 'sqlite:///library.db'

++
||
++
++

### Create VolunteeringPositions Table ###

In [62]:
%%sql
DROP TABLE IF EXISTS VolunteeringPositions;
CREATE TABLE VolunteeringPositions (
    positionID INTEGER PRIMARY KEY AUTOINCREMENT,
    positionName TEXT NOT NULL,
    positionDescription TEXT,
    location TEXT,
    isAvailable INTEGER DEFAULT 1 CHECK(isAvailable IN (0,1))
);

Running query in 'sqlite:///library.db'

++
||
++
++

### Create HelpRequest Table ###

In [64]:
%%sql
DROP TABLE IF EXISTS HelpRequest; 
CREATE TABLE HelpRequest(
    requestID INTEGER PRIMARY KEY AUTOINCREMENT,
    memberID INTEGER,
    requestDate DATE NOT NULL,
    topic TEXT NOT NULL,
    FOREIGN KEY (memberID) REFERENCES Member(memberID)
);

Running query in 'sqlite:///library.db'

++
||
++
++

### Create Register Relationship ###

In [60]:
%%sql
CREATE TABLE Register (
    memberID INTEGER,
    eventID INTEGER,
    PRIMARY KEY (memberID,eventID),
    FOREIGN KEY (memberID) REFERENCES Member(memberID),
    FOREIGN KEY (eventID) REFERENCES Event(eventID)
);

Running query in 'sqlite:///library.db'

++
||
++
++

### Create Borrows Relationship ###

In [61]:
%%sql
DROP TABLE IF EXISTS Borrows;
CREATE TABLE Borrows (
    borrowID INTEGER PRIMARY KEY AUTOINCREMENT,
    itemID INTEGER,
    memberID INTEGER,
    borrowDate DATE,
    dueDate DATE NOT NULL,
    returnDate DATE,
    fineAmount INTEGER DEFAULT 0,
    FOREIGN KEY (itemID) REFERENCES LibraryItem(itemID),
    FOREIGN KEY (memberID) REFERENCES Member(memberID) 
);

Running query in 'sqlite:///library.db'

++
||
++
++

### Create Volunteer Relationship ###

In [63]:
%%sql
DROP TABLE IF EXISTS Volunteer;
CREATE TABLE Volunteer (
    memberID INTEGER,
    positionID INTEGER UNIQUE,
    PRIMARY KEY (memberID,positionID),
    FOREIGN KEY (memberID) REFERENCES Member(memberID),
    FOREIGN KEY (positionID) REFERENCES VolunteeringPosition(positionID)
)

Running query in 'sqlite:///library.db'

++
||
++
++

### Create Triggers ###

In [65]:
%%sql
CREATE TRIGGER set_item_checked_out
AFTER INSERT ON Borrows
FOR EACH ROW
BEGIN
    UPDATE LibraryItem
    SET itemStatus='Checked-Out'
    WHERE itemID=NEW.itemID;
END;

Running query in 'sqlite:///library.db'

++
||
++
++

In [66]:
%%sql
CREATE TRIGGER set_item_available
AFTER DELETE ON Borrows
FOR EACH ROW
BEGIN
    UPDATE LibraryItem
    SET itemStatus = 'Available'
    WHERE itemID = OLD.itemID;
END;

Running query in 'sqlite:///library.db'

++
||
++
++

In [67]:
%%sql
CREATE TRIGGER ensure_future_item_pending
AFTER INSERT ON FutureItem
FOR EACH ROW
BEGIN
    UPDATE LibraryItem
    SET itemStatus='Pending'
    WHERE itemID=NEW.itemID;
END;

Running query in 'sqlite:///library.db'

++
||
++
++

In [68]:
%%sql
CREATE TRIGGER update_future_item_status
AFTER UPDATE ON FutureItem
FOR EACH ROW
BEGIN
    UPDATE LibraryItem
    SET itemStatus=CASE
        WHEN NEW.approvalStatus='Complete' THEN 'Available'
        WHEN NEW.approvalStatus='Denied' THEN itemStatus
        ELSE 'Pending'
    END
    WHERE itemID=NEW.itemID;

    DELETE FROM FutureItem
    WHERE itemID=NEW.itemID
      AND NEW.approvalStatus='Denied';

    DELETE FROM LibraryItem
    WHERE itemID=NEW.itemID
      AND NEW.approvalStatus='Denied';
END;

Running query in 'sqlite:///library.db'

++
||
++
++

In [69]:
%%sql
CREATE TRIGGER IF NOT EXISTS calculate_fine_on_return
AFTER UPDATE ON Borrows
FOR EACH ROW
WHEN NEW.returnDate IS NOT NULL
BEGIN
    UPDATE Borrows
    SET fineAmount = 
        CASE 
            WHEN julianday(NEW.returnDate)>julianday(NEW.dueDate)
            THEN (julianday(NEW.returnDate)-julianday(NEW.dueDate))*1.25
            ELSE 0
        END
    WHERE itemID = NEW.itemID
      AND memberID = NEW.memberID;
END;

Running query in 'sqlite:///library.db'

++
||
++
++

In [70]:
%%sql
CREATE TRIGGER set_item_available_on_return
AFTER UPDATE OF returnDate ON Borrows
FOR EACH ROW
WHEN NEW.returnDate IS NOT NULL
BEGIN
    UPDATE LibraryItem
    SET itemStatus='Available'
    WHERE itemID=NEW.itemID;
END;

Running query in 'sqlite:///library.db'

++
||
++
++

## Insert Tuples ##

In [71]:
%%sql
INSERT INTO LibraryItem (itemID,title,author,releaseYear,issueNumber,itemType,itemStatus)
VALUES 
  (1,'The Great Gatsby','F. Scott Fitzgerald',1925,NULL,'Print Book','Available'),
  (2,'1984','George Orwell',1949,NULL,'Print Book','Available'),
  (3,'To Pimp A Butterfly','Kendrick Lamar',2015,NULL,'Record','Checked-Out'),
  (4,'To Kill A Mockingbird','Harper Lee',1960,NULL,'Print Book','On-Hold'),
  (5,'National Geographic',NULL,2022,10,'Magazine','Available'),
  (6,'Nature Journal','Various',2021,205,'Scientific Journal','Available'),
  (7,'The Beatles: Abbey Road','The Beatles',1969, NULL,'Record','Available'),
  (8,'The Beautiful and Damned','F. Scott Fitzgerald',1922,NULL,'Print Book','Checked-Out'),
  (9,'The Godfather Part II','Francis Ford Coppola',1974,NULL,'CD','Available'),
  (10,'Hamlet','William Shakespeare',1623,NULL,'Print Book','Available');

Running query in 'sqlite:///library.db'

10 rows affected.

++
||
++
++

In [72]:
%%sql
INSERT INTO LibraryItem (itemID,title,author,releaseYear,issueNumber,itemType,itemStatus)
VALUES (55,'ABC','DEF',3093,NULL,'A','Available');

Running query in 'sqlite:///library.db'

1 rows affected.

++
||
++
++

In [73]:
%%sql
INSERT INTO Member (memberID, firstName, lastName, phoneNumber, email, membershipType)
VALUES 
  (1, 'John','Doe','555-0100', 'john.doe@example.com', 'Gold'),
  (2, 'Jane','Doe', '555-0101', 'jane.smith@example.com', 'Silver'),
  (3, 'Alice', 'Brown', NULL, 'alice.brown@example.com', 'Gold'),
  (4, 'Bob', 'Jones', '555-0103', NULL, 'Bronze'),
  (5, 'Charlie', 'Davis', '555-0104', 'charlie.davis@example.com', 'Silver'),
  (6, 'Diana', 'Evans', NULL, 'diana.evans@example.com', 'Gold'),
  (7, 'Ethan', 'Wright', '555-0106', 'ethan.wright@example.com', 'Bronze'),
  (8, 'Fiona', 'Hill', '555-0107', 'fiona.hill@example.com', 'Silver'),
  (9, 'George', 'King', '555-0108', 'george.king@example.com', 'Gold'),
  (10, 'Hannah', 'Lee', NULL, 'hannah.lee@example.com', 'Bronze');

Running query in 'sqlite:///library.db'

10 rows affected.

++
||
++
++

In [74]:
%%sql
INSERT INTO Personnel (personnelID, firstName, lastName, personnelRole, primaryLocation, salary)
VALUES 
  (1, 'Sarah', 'Connor', 'Librarian', 'Downtown', 45000.00),
  (2, 'Mike', 'Tyson', 'Security', 'Uptown', 38000.00),
  (3, 'Linda', 'Smith', 'Manager', 'Main Branch', 60000.00),
  (4, 'Robert', 'Brown', 'Clerk', 'Downtown', 32000.00),
  (5, 'Emily', 'White', 'Assistant', 'Main Branch', 29000.00),
  (6, 'James', 'Johnson', 'Technician', 'Uptown', 35000.00),
  (7, 'Patricia', 'Williams', 'Custodian', 'Downtown', 28000.00),
  (8, 'David', 'Jones', 'Librarian', 'Main Branch', 47000.00),
  (9, 'Barbara', 'Miller', 'IT Specialist', 'Uptown', 55000.00),
  (10, 'Richard', 'Davis', 'Security', 'Main Branch', 40000.00);


Running query in 'sqlite:///library.db'

10 rows affected.

++
||
++
++

In [75]:
%%sql
INSERT INTO Event (eventID, eventName, eventType, eventDate, eventLocation, audienceType, roomNumber)
VALUES 
  (1, 'Book Club Meeting', 'Book Club', '2025-03-15', 'Library Conference Room', 'Adults', 'R101'),
  (2, 'Film Screening: Classic Movies', 'Film Screening', '2025-04-01', 'Main Hall', 'General', 'R102'),
  (3, 'Art Show', 'Exhibition', '2025-05-05', 'Gallery Room', 'Adults', 'R103'),
  (4, 'Children''s Story Time', 'Storytelling', '2025-03-20', 'Kids Room', 'Children', 'R104'),
  (5, 'Poetry Reading', 'Literary', '2025-04-10', 'Library Auditorium', 'Adults', 'R105'),
  (6, 'Science Workshop', 'Workshop', '2025-05-15', 'Tech Room', 'Teens', 'R106'),
  (7, 'Historical Lecture', 'Lecture', '2025-06-01', 'Lecture Hall', 'Adults', 'R107'),
  (8, 'Film Screening: Indie Films', 'Film Screening', '2025-06-15', 'Main Hall', 'General', 'R102'),
  (9, 'Cooking Demonstration', 'Workshop', '2025-07-01', 'Kitchen', 'General', 'R108'),
  (10, 'Music Concert', 'Concert', '2025-07-15', 'Auditorium', 'Adults', 'R109');


Running query in 'sqlite:///library.db'

10 rows affected.

++
||
++
++

In [76]:
%%sql
INSERT INTO LibraryItem (itemID, title, author, releaseYear, issueNumber, itemType, itemStatus)
VALUES 
  (11, 'To Kill a Mockingbird', 'Harper Lee', 1960, NULL, 'Print Book', 'Available'),
  (12, '1984', 'George Orwell', 1949, NULL, 'Print Book', 'Available'),
  (13, 'The Hip Hop Years', 'Alex Ogg', 1999, NULL, 'Print Book', 'Available'),
  (14, 'Illmatic', 'Nas', 1994, NULL, 'CD', 'Available'),
  (15, 'The Chronic', 'Dr. Dre', 1992, NULL, 'CD', 'Available'),
  (16, 'Enter the Wu-Tang (36 Chambers)', 'Wu-Tang Clan', 1993, NULL, 'CD', 'Available'),
  (17, 'Ready to Die', 'The Notorious B.I.G.', 1994, NULL, 'CD', 'Available'),
  (18, 'A Tribe Called Quest: The Anthology', 'A Tribe Called Quest', 1999, NULL, 'CD', 'Available'),
  (19, 'Books and Rhymes: Hip Hop in Literature', 'Various Authors', 2010, NULL, 'Print Book', 'Available'),
  (20, 'The Miseducation of Lauryn Hill', 'Lauryn Hill', 1998, NULL, 'CD', 'Available');

Running query in 'sqlite:///library.db'

10 rows affected.

++
||
++
++

In [77]:
%config SqlMagic.displaylimit = None

displaylimit: Value None will be treated as 0 (no limit)

In [78]:
%%sql
INSERT INTO LibraryItem (itemID, title, author, releaseYear, issueNumber, itemType, itemStatus)
VALUES 
  (74, 'To Kill a Mockingbird', 'Harper Lee', 1960, NULL, 'Print Book', 'Available')

Running query in 'sqlite:///library.db'

1 rows affected.

++
||
++
++

In [79]:
%%sql
INSERT INTO FutureItem (itemID, dateOfArrival, approvalStatus)
VALUES
(11, '2026-08-01', 'Pending'),
(12, '2026-08-10', 'Pending'),
(13, '2026-08-20', 'Complete'),
(14, '2026-09-01', 'Denied'),
(15, '2026-09-15', 'Pending'),
(16, '2026-10-01', 'Complete'),
(17, '2026-10-10', 'Pending'),
(18, '2026-10-15', 'Pending'),
(19, '2026-11-01', 'Pending'),
(20, '2026-11-15', 'Denied');

Running query in 'sqlite:///library.db'

10 rows affected.

++
||
++
++

In [80]:
%%sql
INSERT INTO Register (memberID, eventID)
VALUES
(1, 1),
(2, 2),
(3, 3),
(4, 4),
(5, 5),
(6, 6),
(7, 7),
(8, 8),
(9, 9),
(10, 10);

Running query in 'sqlite:///library.db'

10 rows affected.

++
||
++
++

In [81]:
%%sql
INSERT INTO Borrows (itemID, memberID, borrowDate, dueDate, fineAmount)
VALUES
(1, 1, '2025-03-01', '2025-03-15', 0.0),
(2, 2, '2025-03-05', '2025-03-20', 0.0),
(3, 3, '2025-03-10', '2025-03-25', 5.0),
(4, 4, '2025-03-12', '2025-03-27', 0.0),
(5, 5, '2025-03-13', '2025-03-28', 0.0),
(6, 6, '2025-03-14', '2025-03-29', 0.0),
(7, 7, '2025-03-15', '2025-03-30', 2.5),
(8, 8, '2025-03-16', '2025-03-31', 0.0),
(9, 9, '2025-03-17', '2025-04-01', 0.0),
(10, 10, '2025-03-18', '2025-04-02', 0.0);

Running query in 'sqlite:///library.db'

10 rows affected.

++
||
++
++

In [82]:
%%sql
INSERT INTO VolunteeringPositions (positionName, positionDescription, location, isAvailable)
VALUES
('Reading Buddy', 'Help children improve reading skills', 'Children’s Room', 1),
('Book Organizer', 'Reshelf and organize returned books', 'Main Branch', 1),
('Event Assistant', 'Assist with setup and guest support during events', 'Auditorium', 1),
('Tech Guide', 'Help patrons use public computers and printers', 'Computer Lab', 1),
('Storytime Host', 'Read books aloud to children during story hour', 'Children’s Room', 1),
('Language Partner', 'Converse with newcomers to improve English skills', 'Language Centre', 1),
('Film Screening Usher', 'Assist in seating and guiding attendees', 'Event Hall', 0),
('Art Exhibit Helper', 'Support local artists during setup and takedown', 'Exhibit Space', 1);


Running query in 'sqlite:///library.db'

8 rows affected.

++
||
++
++

In [83]:
%%sql
INSERT INTO Volunteer (memberID, positionID)
VALUES
(1, 1)

Running query in 'sqlite:///library.db'

1 rows affected.

++
||
++
++

In [84]:
%%sql
INSERT INTO Borrows (itemID, memberID, borrowDate, dueDate)
VALUES (101, 1, '2025-03-01', '2025-03-10');

Running query in 'sqlite:///library.db'

1 rows affected.

++
||
++
++

In [85]:
%%sql
SELECT fineAmount FROM Borrows WHERE itemID = 101 AND memberID = 1;
-- Expected: 0.0


Running query in 'sqlite:///library.db'

fineAmount
0


In [86]:
%%sql
UPDATE Borrows
SET returnDate = '2025-03-10' -- '2025-03-15'
WHERE itemID = 101 AND memberID = 1;

Running query in 'sqlite:///library.db'

1 rows affected.

++
||
++
++

In [87]:
%%sql
SELECT fineAmount, borrowDate, dueDate, returnDate
FROM Borrows
WHERE itemID = 101 AND memberID = 1;

Running query in 'sqlite:///library.db'

fineAmount,borrowDate,dueDate,returnDate
0,2025-03-01,2025-03-10,2025-03-10
